# Inverse Intersections for Boolean Satisfiability Problems

Code implementation of the paper "Inverse Intersections for Boolean Satisfiability Problems" by Homer [arXiv](https://arxiv.org/abs/2501.03281). Code generated using GPT-5.

In [1]:
# =============================================================================
# Inverse-Intersections SAT Solver + Robust DIMACS/SATLIB Integration (Jupyter)
# =============================================================================

from collections import deque
from typing import List, Optional, Tuple
from dataclasses import dataclass
import os, gzip, time, glob

# T/F/U symbols for the row (variable) states in a clause-vector
T, F, U = "T", "F", "U"

# =============================================================================
# SymPy → CNF (integer clause list) and CNF → row-matrix (paper §2 notation)
# =============================================================================

from sympy import symbols
from sympy.logic.boolalg import And, Or, Not, BooleanTrue, BooleanFalse, to_cnf

def sympy_to_clauses(phi, var_order=None) -> Tuple[List[List[int]], List[str]]:
    """
    Convert a SymPy Boolean formula to CNF clauses suitable for the algorithm.

    Output:
      - clauses: List of clauses; each clause is a list of ints:
                 +i encodes x_i, -i encodes ¬x_i.
      - var_names: fixed variable order used everywhere downstream.
    """
    cnf = to_cnf(phi, simplify=True)

    # Fix a stable variable order (default: lexical by name)
    if var_order is None:
        syms = sorted(list(cnf.free_symbols), key=lambda s: s.name)
    else:
        name2sym = {s.name: s for s in cnf.free_symbols}
        syms = [name2sym[n] for n in var_order]
    var_names = [s.name for s in syms]
    name_to_index = {n: i + 1 for i, n in enumerate(var_names)}

    # Degenerate cases: treat True/False at formula level
    if cnf is BooleanTrue:
        return [[]], var_names
    if cnf is BooleanFalse:
        return [], var_names  # empty set of clauses signals UNSAT

    def lit_to_int(lit):
        if isinstance(lit, Not):
            return -name_to_index[lit.args[0].name]
        return +name_to_index[lit.name]

    # Ensure clause list shape (And-of-Ors or single Or/unit)
    clause_nodes = list(cnf.args) if isinstance(cnf, And) else [cnf]
    clauses: List[List[int]] = []

    for node in clause_nodes:
        if node is BooleanTrue:
            continue  # clause always-true → skip
        if node is BooleanFalse:
            return [], var_names  # UNSAT
        if isinstance(node, Or):
            lits = []
            for a in node.args:
                if a is BooleanTrue:
                    lits = None
                    break
                if a is BooleanFalse:
                    continue
                lits.append(lit_to_int(a))
            if lits is not None:
                clauses.append(lits)
        else:
            # Unit clause
            clauses.append([lit_to_int(node)])

    return clauses, var_names


def clauses_to_row_matrix(clauses: List[List[int]], num_vars: int) -> List[List[str]]:
    """
    Build the V×C row-matrix R_M over {T,F,U} (paper §2) from integer clauses.
    Column = clause; Row i encodes the literal sign of x_i in that clause (or U).
    """
    num_clauses = len(clauses)
    row_matrix = [[U for _ in range(num_clauses)] for _ in range(num_vars)]
    for j, clause in enumerate(clauses):
        for lit in clause:
            var_idx = abs(lit) - 1
            row_matrix[var_idx][j] = T if lit > 0 else F
    return row_matrix


# =============================================================================
# Core primitives (paper §3.5 “Inverse Intersections”)
# =============================================================================

def overlap_vectors(v1: List[str], v2: List[str]) -> Optional[List[str]]:
    """
    OVERLAP: Intersect two clause-vectors/subtrees.
    """
    num_vars = len(v1)
    merged = [U] * num_vars
    for i in range(num_vars):
        a, b = v1[i], v2[i]
        if a == b:
            merged[i] = a
        elif a == U:
            merged[i] = b
        elif b == U:
            merged[i] = a
        else:
            # a and b are T/F and disagree
            return None
    return merged


def inverse_clause_decomposition(clause_vector: List[str]) -> List[List[str]]:
    """
    INVERSE: Disjoint satisfying decomposition of a single clause-vector.
    """
    num_vars = len(clause_vector)
    literal_positions = [i for i, v in enumerate(clause_vector) if v != U]
    parts: List[List[str]] = []

    for idx in literal_positions:
        part = [U] * num_vars
        # Earlier literals are forced opposite so they cannot satisfy before idx
        for j in literal_positions:
            if j < idx:
                part[j] = T if clause_vector[j] == F else F
        # idx satisfies the clause
        part[idx] = clause_vector[idx]
        parts.append(part)

    # If clause had no literals (all U), there are no satisfying assignments.
    return parts


def build_inverse_sets(row_matrix: List[List[str]]) -> Optional[List[List[List[str]]]]:
    """
    Build per-clause 'inverse' families (paper’s decomposition per column).
    """
    num_vars = len(row_matrix)
    num_clauses = len(row_matrix[0]) if num_vars else 0
    inverse_families: List[List[List[str]]] = []

    for j in range(num_clauses):
        clause_vector = [row_matrix[i][j] for i in range(num_vars)]
        parts = inverse_clause_decomposition(clause_vector)
        if not parts:  # all-U clause → unsatisfiable formula
            return None
        inverse_families.append(parts)

    return inverse_families


# =============================================================================
# Solver front-ends
# =============================================================================

def solve_one_inverse_intersections(row_matrix: List[List[str]]) -> Optional[List[bool]]:
    """
    Find a single satisfying assignment using the inverse-intersections scheme.
    """
    inverse_families = build_inverse_sets(row_matrix)
    if inverse_families is None:
        return None

    queue = deque([part for fam in inverse_families for part in fam])

    while queue:
        candidate = queue.popleft()
        current = candidate
        feasible = True

        for fam in inverse_families:
            overlaps = [m for part in fam if (m := overlap_vectors(current, part)) is not None]
            if not overlaps:
                feasible = False
                break
            # Continue with one overlap; enqueue the rest to explore later
            current = overlaps[0]
            for extra in overlaps[1:]:
                queue.append(extra)

        if feasible:
            # Concretize U’s by choosing False (arbitrary; any choice works)
            return [(v == T) for v in current]

    return None


def solve_all_inverse_intersections(row_matrix: List[List[str]]) -> List[List[bool]]:
    """
    Enumerate all satisfying assignments (exponential in general).
    """
    inverse_families = build_inverse_sets(row_matrix)
    if inverse_families is None:
        return []

    queue = deque([part for fam in inverse_families for part in fam])
    seen_partial = set()
    partial_solutions: List[List[str]] = []

    # Phase 1: collect all maximal overlaps (possibly with U’s)
    while queue:
        candidate = queue.popleft()
        current = candidate
        for fam in inverse_families:
            overlaps = [m for part in fam if (m := overlap_vectors(current, part)) is not None]
            if not overlaps:
                break
            current = overlaps[0]
            for extra in overlaps[1:]:
                queue.append(extra)
        else:
            t = tuple(current)
            if t not in seen_partial:
                seen_partial.add(t)
                partial_solutions.append(current)

    # Phase 2: expand U’s to concrete booleans, then de-duplicate
    def expand_undetermined(vec: List[str]) -> List[List[bool]]:
        out = [[]]
        for v in vec:
            if v == U:
                out = [r + [False] for r in out] + [r + [True] for r in out]
            else:
                out = [r + [v == T] for r in out]
        return out

    expanded = []
    for vec in partial_solutions:
        expanded.extend(expand_undetermined(vec))

    dedup, seen = [], set()
    for a in expanded:
        t = tuple(a)
        if t not in seen:
            seen.add(t)
            dedup.append(a)
    return dedup


# =============================================================================
# Utilities
# =============================================================================

def check_satisfied_sympy(phi, assignment: List[bool], var_names: List[str]) -> bool:
    """
    Substitute a boolean assignment into a SymPy formula and evaluate to Python bool.
    """
    env = {symbols(n): assignment[i] for i, n in enumerate(var_names)}
    return bool(phi.subs(env))


# =============================================================================
# DIMACS/SATLIB Integration (robust parser that ignores % comments)
# =============================================================================

def _open_text(path: str):
    """
    Open .cnf or .cnf.gz as text.
    """
    if path.endswith(".gz"):
        return gzip.open(path, "rt", encoding="utf-8", errors="ignore")
    return open(path, "r", encoding="utf-8", errors="ignore")

def parse_dimacs(path: str) -> Tuple[int, List[List[int]]]:
    """
    Robust DIMACS CNF parser.

    Returns:
        (num_vars, clauses)
    Handles:
      - full-line comments starting with 'c' or 'C'
      - '%' end markers and *inline* '%' comments (common in SATLIB)
      - header 'p cnf <num_vars> <num_clauses>' (tolerant to spacing/case)
      - clauses possibly spanning multiple lines; '0' terminates a clause
      - .cnf and .cnf.gz files
    """
    num_vars: Optional[int] = None
    clauses: List[List[int]] = []
    cur: List[int] = []

    with _open_text(path) as f:
        for raw in f:
            line = raw.strip()
            if not line:
                continue

            # Strip inline '%' comments or trailing '%' lines
            if '%' in line:
                line = line.split('%', 1)[0].strip()
                if not line:
                    continue

            # Skip full-line comments
            if line.startswith('c') or line.startswith('C'):
                continue

            # Header
            if line.startswith('p') or line.startswith('P'):
                parts = line.split()
                # find 'cnf' token (any case)
                try:
                    i_cnf = next(i for i, p in enumerate(parts) if p.lower() == 'cnf')
                except StopIteration:
                    # non-standard header, ignore
                    continue
                if len(parts) >= i_cnf + 3:
                    try:
                        num_vars = int(parts[i_cnf + 1])
                        # num_clauses_hdr = int(parts[i_cnf + 2])  # unused
                    except ValueError:
                        pass
                continue

            # Clause content
            for tok in line.split():
                if tok == '0':
                    if cur:
                        clauses.append(cur)
                        cur = []
                    continue
                if not tok:
                    continue
                try:
                    lit = int(tok)
                except ValueError:
                    # If garbage appears after a clause (rare), skip rest of the line
                    break
                if lit == 0:
                    if cur:
                        clauses.append(cur)
                        cur = []
                else:
                    cur.append(lit)

    # Flush if file didn't end with '0'
    if cur:
        clauses.append(cur)

    # Infer num_vars if header missing
    if num_vars is None:
        num_vars = max((abs(l) for cl in clauses for l in cl), default=0)

    return num_vars, clauses

def clauses_to_row_matrix_from_dimacs(num_vars: int, clauses: List[List[int]]) -> List[List[str]]:
    return clauses_to_row_matrix(clauses, num_vars)

def evaluate_clause(clause: List[int], assignment: List[bool]) -> bool:
    for lit in clause:
        v = assignment[abs(lit) - 1]
        if (lit > 0 and v) or (lit < 0 and not v):
            return True
    return False

def evaluate_cnf(clauses: List[List[int]], assignment: List[bool]) -> bool:
    return all(evaluate_clause(cl, assignment) for cl in clauses)

@dataclass
class SATResult:
    path: str
    num_vars: int
    num_clauses: int
    sat: bool
    assignment: Optional[List[bool]]
    elapsed_s: float
    verified: Optional[bool]  # None if not checked

def solve_one_dimacs(path: str, verify: bool = True) -> SATResult:
    V, clauses = parse_dimacs(path)
    RM = clauses_to_row_matrix_from_dimacs(V, clauses)
    t0 = time.perf_counter()
    sol = solve_one_inverse_intersections(RM)
    dt = time.perf_counter() - t0

    if sol is None:
        return SATResult(path, V, len(clauses), False, None, dt, None)

    verified = evaluate_cnf(clauses, sol) if verify else None
    return SATResult(path, V, len(clauses), True, sol, dt, verified)

def expand_glob_or_dir(path_or_glob: str) -> List[str]:
    if os.path.isdir(path_or_glob):
        files = sorted(
            glob.glob(os.path.join(path_or_glob, "*.cnf")) +
            glob.glob(os.path.join(path_or_glob, "*.cnf.gz"))
        )
    else:
        files = sorted(glob.glob(path_or_glob))
    return files

from tqdm import tqdm

def batch_run(path_or_glob: str, limit: Optional[int] = None, verify: bool = True) -> List[SATResult]:
    files = expand_glob_or_dir(path_or_glob)
    if limit is not None:
        files = files[:limit]
    results: List[SATResult] = []
    for p in tqdm(files):
        try:
            results.append(solve_one_dimacs(p, verify=verify))
        except Exception as e:
            results.append(SATResult(p, 0, 0, False, None, 0.0, False))
            print(f"[ERROR] {p}: {e}")
    return results

# Reproducing test instance from paper

In [2]:
# =============================================================================
# Demo: paper’s test instance R (section 2 / answer set in §2.5)
# =============================================================================

# Define variables
x1, x2, x3, x4 = symbols('x1 x2 x3 x4')

# R = (¬x1 ∨ x3 ∨ x4) ∧ (x2) ∧ (x1 ∨ ¬x2 ∨ ¬x3 ∨ x4) ∧ (¬x2 ∨ ¬x4)
R_phi = And(
    Or(Not(x1), x3, x4),
    x2,
    Or(x1, Not(x2), Not(x3), x4),
    Or(Not(x2), Not(x4))
)

# SymPy → clauses → row-matrix
clauses, var_names = sympy_to_clauses(R_phi)
V = len(var_names)
row_matrix = clauses_to_row_matrix(clauses, V)

# Solve (one and all)
one_solution = solve_one_inverse_intersections(row_matrix)
all_solutions = solve_all_inverse_intersections(row_matrix)

# Pretty-print
def show(sol):
    return {var_names[i]: sol[i] for i in range(len(sol))}

print("Variable order:", var_names)
print("One solution:", show(one_solution) if one_solution is not None else None)
print("All solutions:")
for s in all_solutions:
    print(show(s))

# Sanity check against the SymPy formula
assert all(check_satisfied_sympy(R_phi, s, var_names) for s in all_solutions)

Variable order: ['x1', 'x2', 'x3', 'x4']
One solution: {'x1': True, 'x2': True, 'x3': True, 'x4': False}
All solutions:
{'x1': True, 'x2': True, 'x3': True, 'x4': False}
{'x1': False, 'x2': True, 'x3': False, 'x4': False}


# DIMACS / SATLIB

## Smoke Test

In [3]:
# =============================================================================
# (Optional) Tiny smoke test — comment out if not needed
# =============================================================================
# Minimal inline DIMACS to sanity-check:

import os

toy = """c (¬x1 ∨ x3 ∨ x4) ∧ (x2) ∧ (x1 ∨ ¬x2 ∨ ¬x3 ∨ x4) ∧ (¬x2 ∨ ¬x4)
p cnf 4 4
-1 3 4 0
2 0
1 -2 -3 4 0
-2 -4 0
% end
"""
with open("toy.cnf", "w") as f:
    f.write(toy)
r = solve_one_dimacs("toy.cnf", verify=True)
print(r)
if r.assignment:
    print("bits:", "".join("1" if b else "0" for b in r.assignment))
os.remove("toy.cnf")

SATResult(path='toy.cnf', num_vars=4, num_clauses=4, sat=True, assignment=[False, True, False, False], elapsed_s=1.9333994714543223e-05, verified=True)
bits: 0100


## Run a specific SAT instance

In [4]:
from pathlib import Path

path = Path("data/uf20-01.cnf")

res = solve_one_dimacs(str(path))
res, ("".join("1" if b else "0" for b in res.assignment) if res.assignment else None)

(SATResult(path='data/uf20-01.cnf', num_vars=20, num_clauses=91, sat=True, assignment=[False, True, True, True, False, False, False, True, True, True, True, False, False, True, True, False, True, True, True, True], elapsed_s=3.7978627090051305, verified=True),
 '01110001111001101111')

## Run a directory containining SAT instances

In [5]:
results = batch_run("data/*.cnf", limit=20, verify=True)

for result in results:
  print(result)

100%|██████████| 5/5 [00:41<00:00,  8.35s/it]

SATResult(path='data/uf20-01.cnf', num_vars=20, num_clauses=91, sat=True, assignment=[False, True, True, True, False, False, False, True, True, True, True, False, False, True, True, False, True, True, True, True], elapsed_s=3.8253121669986285, verified=True)
SATResult(path='data/uf20-02.cnf', num_vars=20, num_clauses=91, sat=True, assignment=[False, False, False, False, True, False, True, True, True, False, False, False, False, True, False, True, False, False, True, False], elapsed_s=1.1708403750089929, verified=True)
SATResult(path='data/uf20-03.cnf', num_vars=20, num_clauses=91, sat=True, assignment=[True, True, True, True, False, True, True, True, True, True, True, False, True, False, False, True, True, True, False, True], elapsed_s=28.741774082998745, verified=True)
SATResult(path='data/uf20-04.cnf', num_vars=20, num_clauses=91, sat=True, assignment=[True, False, True, True, False, False, True, False, False, True, True, False, True, False, False, True, True, False, False, False], e